## One-hot encoding vs target encoding
Using consumer loan application data, I show below that we can often extract more information from categorical data for a machine learning model with target encoding compared to one-hot encoding.

In [64]:
# Load libraries
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [65]:
# Load and explore data
raw = pd.read_csv('https://raw.githubusercontent.com/sedeh/Datasets/main/loan_200k.csv')
print(raw.shape)
print(raw.columns)

(200000, 18)
Index(['id', 'member_id', 'loan_amnt', 'term_in_months', 'interest_rate',
       'payment', 'grade', 'sub_grade', 'employment_length', 'home_owner',
       'income', 'verified', 'default', 'purpose', 'zip_code', 'addr_state',
       'open_accts', 'credit_debt'],
      dtype='object')


In [69]:
# Select data
col = 'sub_grade'
# col = 'grade'
df = raw.select_dtypes('number').drop(['id', 'member_id'], axis=1)
df[col] = raw[col]
print(df[col].nunique())

7


In [70]:
# One-hot encoding
data = pd.get_dummies(df, columns=[col], drop_first=True)
y = data['interest_rate']
X = data.drop(['interest_rate'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
model = XGBRegressor()
model.fit(X_train, y_train)
y_true = model.predict(X_test)
mean_absolute_error(y_true, y_test)

[01:34:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


1.1119605288410184

In [71]:
# Target encoding
means = df[[col, 'interest_rate']].groupby(col).mean().reset_index()
keys = means[col]
values = means.interest_rate
mapper = {keys[i]: values[i] for i in range(len(keys))}
df['interest_rate_by_grade'] = df[col].map(mapper)
data = df.drop(col, axis=1)
y = data['interest_rate']
X = data.drop(['interest_rate'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
model = XGBRegressor()
model.fit(X_train, y_train)
y_true = model.predict(X_test)
mean_absolute_error(y_true, y_test)

[01:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


1.0050220181236267